## Data

In [ ]:
from data import get_data_paths, celeb2mask, CustomDataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
import torch

from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
%%time
# get data paths
data_paths = get_data_paths()
# data_paths = [i for i in data_paths if i[0] == "community_dataset"]
# data_paths = [i for i in data_paths if i[0] != "community_dataset"]
np.random.shuffle(data_paths)
data_paths = data_paths[:100]

# split data
train_paths, test_paths = train_test_split(data_paths, test_size=0.2, random_state=42)
test_paths, val_paths = train_test_split(test_paths, test_size=0.5, random_state=42)

print(f"Train: {len(train_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}")

# Datasets
train_dataset = CustomDataset(train_paths)
val_dataset = CustomDataset(val_paths)
test_dataset = CustomDataset(test_paths)

# Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

## Training

In [ ]:
from pretrained_models import PretrainedModel
import pytorch_lightning as pl
from shared import get_callbacks

In [ ]:
test_metrics = pd.DataFrame({"model":[], "backbone": [], "optimizer": [], "loss":[],
                             "IoU":[], "F1":[], "Inference Time":[]})

In [ ]:
# utils
epochs = 1
weight = [  2.3083, 107.5944, 255.0750,  53.6322,   4.2954, 142.1090, 410.2553,
          3.6461,  91.0078, 304.7269, 243.5046]
weight = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
weight = [0.5, 3, 3, 1, 0.5, 3, 2, 0.5, 1.5, 1.5, 1.5]
assert len(weight) == 11

loss_list = ["BCEWeighted", "DiceLoss", "DiceBCE", "FocalLoss",  "DiceFocal", "BiasLoss"]
backbones = ["timm-efficientnet-b1", "timm-mobilenetv3_large_075"]
optimizers_list = ["Adam", "AdamP", "MADGRAD"]

### Unet

In [ ]:
arch = "Unet"

for encoder_name in backbones:
    for loss_name in loss_list:
        for optimizer_name in optimizers_list:

            print(encoder_name, loss_name, optimizer_name)

            # init model
            params = {"weight":weight, "optimizer_name":optimizer_name,
                      "lr": 1e-4, "weight_decay": 0,
                      "loss_name": loss_name, "arch":arch,
                      "encoder_name":encoder_name}
            callbacks, logger = get_callbacks(params)
            model = PretrainedModel(**params)

            # train model
            trainer = pl.Trainer(
            gpus=1, 
            max_epochs=epochs,
            logger=logger,
            callbacks=callbacks,
            )

            trainer.fit(
                model, 
                train_dataloaders=train_dataloader, 
                val_dataloaders=val_dataloader,
            )

            # evaluate
            start_time = time.time()
            model_metrics = trainer.test(model, test_dataloader)
            end_time = time.time() - start_time

            test_metrics = test_metrics.append({"model":arch, "backbone": encoder_name, "loss": loss_name,
                                                "optimizer": optimizer_name,
                                                "IoU":model_metrics[0]["test_iou"], 
                                                "F1":model_metrics[0]["test_f1"],
                                                "Inference Time":end_time / test_dataset.__len__() * 1000},
                                               ignore_index=True)
            # save model
            torch.save(model.state_dict(), f"../data/models/{arch}_{encoder_name}.pt")

        
            del model

### DeepLabV3

In [ ]:
arch = "deeplabv3plus"

for encoder_name in backbones:
    for loss_name in loss_list:
        for optimizer_name in optimizers_list:

            print(encoder_name, loss_name, optimizer_name)

            # init model
            params = {"weight":weight, "optimizer_name":optimizer_name,
                      "lr": 1e-4, "weight_decay": 0,
                      "loss_name": loss_name, "arch":arch,
                      "encoder_name":encoder_name}
            callbacks, logger = get_callbacks(params)
            model = PretrainedModel(**params)

            # train model
            trainer = pl.Trainer(
            gpus=1, 
            max_epochs=epochs,
            logger=logger,
            callbacks=callbacks,
            )

            trainer.fit(
                model, 
                train_dataloaders=train_dataloader, 
                val_dataloaders=val_dataloader,
            )

            # evaluate
            start_time = time.time()
            model_metrics = trainer.test(model, test_dataloader)
            end_time = time.time() - start_time

            test_metrics = test_metrics.append({"model":arch, "backbone": encoder_name, "loss": loss_name,
                                                "optimizer": optimizer_name,
                                                "IoU":model_metrics[0]["test_iou"], 
                                                "F1":model_metrics[0]["test_f1"],
                                                "Inference Time":end_time / test_dataset.__len__() * 1000},
                                               ignore_index=True)
            # save model
            torch.save(model.state_dict(), f"../data/models/{arch}_{encoder_name}.pt")
        
            del model